In [1]:
import torch
import pickle
import os
import snntorch as snn
import torch.nn as nn
import numpy as np
from collections import OrderedDict

# PyTorch Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Function
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.optim import Adam
from torch.utils.data import random_split
from torch.utils.data import DataLoader, random_split
import torchvision

# Additional Imports
import snntorch as snn
import matplotlib.pyplot as plt
import numpy as np
import time
import os

# Dataset
import tonic
import tonic.transforms as transforms
from torch.utils.data import DataLoader
from tonic import DiskCachedDataset

# Network
import snntorch as snn
from snntorch import surrogate
from snntorch import functional as SF
from snntorch import spikeplot as splt
from snntorch import utils

import torchvision.transforms as torchvision_transforms
import tonic
import tonic.transforms as transforms

In [2]:
# Define sensor size for NMNIST dataset
sensor_size = tonic.datasets.NMNIST.sensor_size

# Define transformations
# Note: The use of torch.from_numpy is removed as Tonic's transforms handle conversion.
transform = tonic.transforms.Compose([
    transforms.Denoise(filter_time=10000),
    transforms.ToFrame(sensor_size=sensor_size, time_window=10000),
    # torchvision.transforms.RandomRotation is not directly applicable to event data.
    # If rotation is needed, it should be done on the frames after conversion by ToFrame.
])

# Load NMNIST datasets without caching
trainset = tonic.datasets.NMNIST(save_to='./tmp/data', transform=transform, train=True)
testset = tonic.datasets.NMNIST(save_to='./tmp/data', transform=transform, train=False)

# Split trainset into training and validation datasets
train_size = int(0.8 * len(trainset))
val_size = len(trainset) - train_size
train_dataset, val_dataset = random_split(trainset, [train_size, val_size])

# Create DataLoaders for training, validation, and testing
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=tonic.collation.PadTensors(batch_first=False))
val_loader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False))
test_loader = DataLoader(testset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False))

# Fetch a single batch from the train_loader to inspect the shape
data, targets = next(iter(train_loader))
print(f"Data shape: {data.shape}")  # Example output: torch.Size([batch_size, timesteps, channels, height, width])
print(f"Targets shape: {targets.shape}")  # Example output: torch.Size([batch_size])


Data shape: torch.Size([31, 128, 2, 34, 34])
Targets shape: torch.Size([128])


In [3]:
config = {
    # SNN
    "threshold1": 2.5,
    "threshold2": 8.0,
    "threshold3": 4.0,
    "threshold4": 2.0,
    "beta": 0.5,
    "num_steps": 10,
    
    # SNN Dense Shape
    "dense1_input": 2312,
    "num_classes": 10,
    

    # Network
    "batch_norm": True,
    "dropout": 0.3,

    # Hyper Params
    "lr": 0.007,

    # Early Stopping
    "min_delta": 1e-6,
    "patience_es": 20,

    # Training
    "epochs": 2
}

In [4]:
class SNN_tests(nn.Module):
  def __init__(self, config):
    super(SNN_tests, self).__init__()

    # Initialize configuration parameters
      # LIF
    self.thresh1 = config["threshold1"]
    self.thresh2 = config["threshold2"]
    self.thresh3 = config["threshold3"]
    self.thresh4 = config["threshold4"]
    self.beta = config["beta"]
    self.num_steps = config["num_steps"]

      # Hyper Params for Layers
    self.batch_norm = config["batch_norm"]
    self.dropout_percent = config["dropout"]

      # Dense Shape
    self.dense1_input = config["dense1_input"]
    self.num_classes = config["num_classes"]

      # Network Layers
    self.fc1 = nn.Linear(self.dense1_input, self.dense1_input//4)
    self.lif1 = snn.Leaky(beta=self.beta, threshold=self.thresh1)
    
    
    self.fc2 = nn.Linear(self.dense1_input//4, self.dense1_input//8)
    self.lif2 = snn.Leaky(beta=self.beta, threshold=self.thresh2)
    
    self.fc3 = nn.Linear(self.dense1_input//8, self.num_classes)
    self.lif3 = snn.Leaky(beta=self.beta, threshold=self.thresh3)
    
    self.flatten = nn.Flatten()
    self.fc4 = nn.Linear(self.dense1_input//8, self.num_classes)
    self.lif4 = snn.Leaky(beta=self.beta, threshold=self.thresh4)
    self.dropout = nn.Dropout(self.dropout_percent)
    
    # Extra (not used)
    self.batch_norm_1 = nn.BatchNorm2d(num_features=16)
    self.batch_norm_2 = nn.BatchNorm2d(num_features=32)
    
    
    # Forward Pass
  def forward(self, inpt):
    mem1 = self.lif1.init_leaky()
    mem2 = self.lif2.init_leaky()
    mem3 = self.lif3.init_leaky()

    all_outputs = {layer: [] for layer in ['inputs', 'fc1_outputs', 'lif1_spikes', 'fc2_outputs', 'lif2_spikes', 'fc3_outputs', 'lif3_spikes', 'mem1', 'mem2', 'mem3']}
    print(inpt.shape)
    for step in range(inpt.shape[0]):  # assuming inpt shape is (batch, time, ...)
        current_input = inpt[step, ...].to(device)
        
        current_input = self.flatten(current_input)

        current1 = self.fc1(current_input)
        spike1, mem1 = self.lif1(current1, mem1)
        current2 = self.fc2(spike1)
        spike2, mem2 = self.lif2(current2, mem2)
        current3 = self.fc3(spike2)
        spike3, mem3 = self.lif3(current3, mem3)

        # Collect outputs for each step
        all_outputs['inputs'].append(current_input.detach().cpu().numpy())
        all_outputs['fc1_outputs'].append(current1.detach().cpu().numpy())
        all_outputs['lif1_spikes'].append(spike1.detach().cpu().numpy())
        all_outputs['fc2_outputs'].append(current2.detach().cpu().numpy())
        all_outputs['lif2_spikes'].append(spike2.detach().cpu().numpy())
        all_outputs['fc3_outputs'].append(current3.detach().cpu().numpy())
        all_outputs['lif3_spikes'].append(spike3.detach().cpu().numpy())
        all_outputs['mem1'].append(mem1.detach().cpu().numpy())
        all_outputs['mem2'].append(mem2.detach().cpu().numpy())
        all_outputs['mem3'].append(mem3.detach().cpu().numpy())

    return all_outputs
  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SNN_tests(config).to(device)

/home/copparihollmann/miniconda3/envs/SNNCpp/lib/python3.12/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [5]:
import pickle

def load_and_save_model_outputs(model_path, test_loader, device):
    model = SNN_tests(config).to(device)
    model.eval()  # Set model to evaluation mode

    # Load the model weights
    model.load_state_dict(torch.load(model_path, map_location=device))

    # Grab a single sample from the loader
    single_sample, _ = next(iter(test_loader))  # Adjust if your dataloader is different
    single_sample = single_sample.to(device)

    # Run the forward pass
    layer_outputs = model(single_sample[:,0, ...].unsqueeze(1))  # Adjust slicing based on your data

    # Save the outputs
    save_layer_outputs(layer_outputs)

def save_layer_outputs(layer_outputs):
    base_dir = "./intermediate_outputs"
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)
    
    for layer_name, timesteps_outputs in layer_outputs.items():
        layer_dir = os.path.join(base_dir, layer_name)
        if not os.path.exists(layer_dir):
            os.makedirs(layer_dir)
        
        for step, output in enumerate(timesteps_outputs):
            file_path = os.path.join(layer_dir, f"{layer_name}_timestep_{step}.bin")
            with open(file_path, 'wb') as f:
                pickle.dump(output, f)

# Assuming the model and device are defined, and model's forward() is correctly returning layer outputs
# Example Usage
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = "/home/copparihollmann/neuroTUM/SpikingC/SpikingCpp/notebooks/best_SNN_model.pth"
load_and_save_model_outputs(model_path, test_loader, device)


torch.Size([31, 1, 2, 34, 34])


In [6]:
class SNN(nn.Module):
  def __init__(self, config):
    super(SNN, self).__init__()

    # Initialize configuration parameters
      # LIF
    self.thresh1 = config["threshold1"]
    self.thresh2 = config["threshold2"]
    self.thresh3 = config["threshold3"]
    self.beta = config["beta"]
    self.num_steps = config["num_steps"]

      # Hyper Params for Layers
    self.batch_norm = config["batch_norm"]
    self.dropout_percent = config["dropout"]

      # Dense Shape
    self.dense1_input = config["dense1_input"]
    self.num_classes = config["num_classes"]

      # Network Layers
    self.fc1 = nn.Linear(self.dense1_input, self.dense1_input//4)
    self.lif1 = snn.Leaky(beta=self.beta, threshold=self.thresh1)
    
    
    self.fc2 = nn.Linear(self.dense1_input//4, self.dense1_input//8)
    self.lif2 = snn.Leaky(beta=self.beta, threshold=self.thresh2)
    
    self.fc3 = nn.Linear(self.dense1_input//8, self.num_classes)
    self.lif3 = snn.Leaky(beta=self.beta, threshold=self.thresh3)
    
    self.flatten = nn.Flatten()
    
    
    # Forward Pass
  def forward(self, inpt):
    mem1 = self.lif1.init_leaky()
    mem2 = self.lif2.init_leaky()
    mem3 = self.lif3.init_leaky()
    #mem4 = self.lif4.init_leaky()

    spike3_rec = []
    mem3_rec = []
    
    #print(inpt.shape)

    for step in range(inpt.shape[0]):
      #print(inpt[step].shape)
      
      current_input = inpt[step]
      current_input = self.flatten(current_input)
      
      current1 = self.fc1(current_input)
      spike1, mem1 = self.lif1(current1, mem1)

      current2 = self.fc2(spike1)
      spike2, mem2 = self.lif2(current2, mem2)

      current3 = self.fc3(spike2)
      spike3, mem3 = self.lif3(current3, mem3)
      
      #current4 = self.fc4(spike3)
      #spike4, mem4 = self.lif4(current4, mem4)

      spike3_rec.append(spike3)
      mem3_rec.append(mem3)

    return torch.stack(spike3_rec, dim=0), torch.stack(mem3_rec, dim=0)

In [7]:
def save_weights_and_biases_to_binary(model_path, save_directory="model_params_binary"):
    # Load the model's state dictionary
    state_dict = torch.load(model_path, map_location=torch.device('cpu'))

    # Create the directory if it doesn't exist
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)

    # Save each parameter to a separate binary file
    for name, param in state_dict.items():
        file_path = os.path.join(save_directory, f"{name.replace('.', '_')}.bin")
        # Open the file in binary write mode and save the tensor data
        with open(file_path, 'wb') as f:
            # Convert the tensor to numpy and write as a binary stream
            numpy_array = param.numpy()
            f.write(numpy_array.tobytes())

# Example usage
model_path = "/home/copparihollmann/neuroTUM/SpikingC/SpikingCpp/notebooks/best_SNN_model.pth"
save_weights_and_biases_to_binary(model_path)

In [ ]:
# Function to save weights and biases as binary files
def save_weights_and_biases_binary(model, directory="model_params"):
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    for name, param in model.named_parameters():
        # Construct file path
        file_path = os.path.join(directory, f"{name.replace('.', '_')}.bin")
        # Open file in binary write mode and save using pickle
        with open(file_path, 'wb') as f:
            pickle.dump(param.detach().cpu().numpy(), f)

# Function to save layer outputs as binary files
def save_layer_outputs_binary(model, single_sample, directory="layer_outputs"):
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # Ensure single_sample is on the correct device and has batch dimension
    single_sample = single_sample.to(device).unsqueeze(0)
    
    # Run the model and get outputs
    model.eval()
    with torch.no_grad():
        layer_outputs = model(single_sample)
    
    # Save each layer's outputs as a binary file
    for layer_name, output in layer_outputs.items():
        # Flatten output and convert to numpy if it's a tensor
        if isinstance(output, torch.Tensor):
            output = output.detach().cpu().numpy().flatten()
        
        # Construct file path
        file_path = os.path.join(directory, f"{layer_name.replace('.', '_')}.bin")
        # Open file in binary write mode and save using pickle
        with open(file_path, 'wb') as f:
            pickle.dump(output, f)

# Example usage:

# Assume 'model' is a PyTorch model and 'single_sample' is a tensor representing a single data sample
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
single_sample = torch.rand((1, 3, 224, 224))  # Example tensor shape for an image
single_sample = single_sample.to(device)

# Save weights and biases as binary files
save_weights_and_biases_binary(model)

# Save layer outputs as binary files
save_layer_outputs_binary(model, single_sample)


In [19]:
def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

def save_layer_outputs_binary(model, input_tensor, device, base_dir="intermediate_outputs"):
    model.eval()
    model.to(device)
    input_tensor = input_tensor.to(device)

    # Prepare the model to save outputs
    saved_outputs = {}

    def hook_fn(module, input, output, name):
        if name not in saved_outputs:
            saved_outputs[name] = []
        saved_outputs[name].append(output)

    hooks = []
    for name, module in model.named_modules():
        if not isinstance(module, nn.Sequential) and not isinstance(module, SNN_tests) and module != model:
            hooks.append(module.register_forward_hook(lambda module, input, output, name=name: hook_fn(module, input, output, name)))

    # Run model forward pass
    _ = model(input_tensor)

    # Remove hooks after use
    for hook in hooks:
        hook.remove()

    # Save outputs to binary files for each timestep
    for name, outputs in saved_outputs.items():
        layer_dir = os.path.join(base_dir, name)
        ensure_dir(layer_dir)
        for timestep, timestep_output in enumerate(outputs):
            file_name = os.path.join(layer_dir, f"output_timestep_{timestep}.bin")
            timestep_output_flat = timestep_output.flatten()
            with open(file_name, 'wb') as f:
                f.write(timestep_output_flat.tobytes())
            print(f"Saved {name} output at timestep {timestep} to {file_name}")

# Example usage
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SNN_tests(config)
input_tensor = torch.randn((30, 2, 34, 34))  # Example input tensor with 30 timesteps

save_layer_outputs_binary(model, input_tensor, device)

torch.Size([30, 2, 34, 34])


AttributeError: 'Tensor' object has no attribute 'tobytes'

In [ ]:
import torch
import numpy as np

def load_binary_file_to_tensor(file_path, dtype=np.float32, shape=(31, 2, 34, 34)):
    # Read the binary file
    with open(file_path, 'rb') as f:
        data = np.fromfile(f, dtype=dtype)

    # Reshape the data to match the input shape
    data = data.reshape(shape)

    # Convert the NumPy array to a PyTorch tensor
    tensor = torch.tensor(data, dtype=torch.float32)
    
    return tensor

# Use the function to load the binary file into a tensor
file_path = "/home/copparihollmann/neuroTUM/SpikingCpp/notebooks/tmp/data/NMNIST/Test/7/00001.bin"
input_tensor = load_binary_file_to_tensor(file_path)

# Now you can process this tensor in a loop for each time step
for timestep in range(input_tensor.shape[0]):
    timestep_data = input_tensor[timestep]  # This selects the data for the current timestep
    # Process the data for the current timestep
    # For example, pass it through your model
    output = model(timestep_data.unsqueeze(0))  # Add batch dimension if needed

    # ... Save or process the output as needed ...



In [ ]:
import torch

# Assume 'weights' is your pre-trained weights tensor
weights = model.fc1.weight.data  # example to get the weights from the first fully connected layer

# Compute the scale and zero-point for quantization
scale = (weights.max() - weights.min()) / 255
zero_point = weights.min()

# Quantize the weights to integers
quantized_weights = torch.round((weights - zero_point) / scale).to(torch.uint8)

# Dequantize back to float to use in computations (simulate quantized operations)
dequantized_weights = quantized_weights.float() * scale + zero_point

# Now replace the original weights with the dequantized weights
model.fc1.weight.data = dequantized_weights


In [ ]:
import numpy as np

def quantize_weights_to_int8(weights):
    """
    Quantizes the given weights to 8-bit integer values.

    :param weights: The floating-point weights of a neural network layer.
    :return: A tuple of (quantized_weights, scale, zero_point), where
        quantized_weights is an ndarray of int8 values representing the quantized weights,
        scale is the scaling factor used for quantization,
        zero_point is the shift used to align the floating point zero to the center of the int8 range.
    """
    # Calculate the range of the weight values
    min_val, max_val = weights.min(), weights.max()
    
    # Calculate scale and zero_point for the quantization
    scale = (max_val - min_val) / 255
    zero_point = round(128 - max_val / scale)

    # Quantize weights to integer values
    quantized_weights = np.round(weights / scale + zero_point).astype(np.int8)

    # Ensure zero_point is set such that 0.0 maps to an integer value of zero
    quantized_weights = np.clip(quantized_weights, -128, 127)

    return quantized_weights, scale, zero_point

# Dummy weights for demonstration
original_weights = np.random.randn(3, 3).astype(np.float32)

# Perform quantization
quantized_weights, scale, zero_point = quantize_weights_to_int8(original_weights)

# Display results
print("Original weights:")
print(original_weights)
print("\nQuantized weights:")
print(quantized_weights)
print("\nScale:", scale)
print("Zero point:", zero_point)


In [ ]:
import torch
import torch.quantization

# Assume 'model' is your pre-trained model
model.eval()

# Specify the quantization configuration
# In this case, we are using static quantization
model.qconfig = torch.quantization.get_default_qconfig('fbgemm')

# Insert observers to collect statistics on the model's weights
torch.quantization.prepare(model, inplace=True)

# Calibrate the model with representative data
with torch.no_grad():
    for input, _ in representative_dataset:
        model(input)  # Use the model's forward method to perform calibration

# Convert the model to a quantized version
torch.quantization.convert(model, inplace=True)

# Now 'model' is quantized, and you can save, load, or run inference with it
